In [ ]:
!pip install kiteconnect gspread oauth2client --quiet


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 771.5/771.5 kB 18.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 247.5/247.5 kB 27.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 51.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.6/74.6 kB 8.7 MB/s eta 0:00:00


In [ ]:
import time
from google.colab import userdata
from kiteconnect import KiteConnect
from requests.exceptions import RequestException
from kiteconnect.exceptions import KiteException

def initialize_kite_with_retry(max_retries=5, retry_delay=15):
    for attempt in range(max_retries):
        try:
            # Retrieve API key, API secret, and access token from Colab's environment
            api_key = userdata.get('api_key')
            api_secret = userdata.get('api_secret')
            access_token = userdata.get('access_token')

            # Initialize KiteConnect with the API key
            kite = KiteConnect(api_key=api_key)

            # Set the access token for your session
            kite.set_access_token(access_token)

            print("Kite API initialized successfully.")
            return kite
        except (KiteException, RequestException) as e:
            print(f"Attempt {attempt + 1} failed with error: {e}")
            time.sleep(retry_delay)  # Wait before retrying

    # After all retries have failed, raise an exception
    raise Exception("Failed to initialize Kite API after multiple retries.")

try:
    # Initialize Kite API with retry logic
    kite = initialize_kite_with_retry()

    # Example: Now you can use `kite` to make API calls, such as fetching the user profile
    profile = kite.profile()
    print(profile)

except Exception as e:
    print(f"An error occurred: {e}")


Kite API initialized successfully.
{'user_id': 'AZ8917', 'user_type': 'individual/res_no_nn', 'email': 'arpiarpithak25@gmail.com', 'user_name': 'Arpitha Kemparaju', 'user_shortname': 'Arpitha', 'broker': 'ZERODHA', 'exchanges': ['BSE', 'BFO', 'CDS', 'BCD', 'NFO', 'MF', 'NSE'], 'products': ['CNC', 'NRML', 'MIS', 'BO', 'CO'], 'order_types': ['MARKET', 'LIMIT', 'SL', 'SL-M'], 'avatar_url': None, 'meta': {'demat_consent': 'consent'}}


In [ ]:
from google.colab import userdata
import json
from oauth2client.service_account import ServiceAccountCredentials
import gspread
import time
import pandas as pd

def with_retry(func):
    """Decorator to add retry logic around function calls that might fail."""
    def wrapper(*args, **kwargs):
        max_retries = 5
        retry_delay = 15  # seconds
        for attempt in range(max_retries):
            try:
                return func(*args, **kwargs)
            except Exception as e:
                if attempt < max_retries - 1:
                    print(f"Attempt {attempt + 1} failed with error: {e}. Retrying in {retry_delay} seconds...")
                    time.sleep(retry_delay)
                else:
                    print(f"Failed after {max_retries} attempts.")
                    raise
    return wrapper

@with_retry
def load_service_account_credentials():
    # Retrieve and load the service account credentials from Colab's environment
    service_account_info_json = userdata.get('json_googlesheets')  # Use the actual name of your stored credentials
    return json.loads(service_account_info_json)

@with_retry
def authenticate_and_get_client(service_account_info):
    # Define the scope for Google Sheets and Drive
    scope = [
        'https://spreadsheets.google.com/feeds',
        'https://www.googleapis.com/auth/drive'
    ]

    # Authenticate using the service account info
    creds = ServiceAccountCredentials.from_json_keyfile_dict(service_account_info, scope)

    # Authorize the client with the credentials
    return gspread.authorize(creds)

@with_retry
def get_sheet_data(client, sheet_name, worksheet_name):
    # Open the Google Sheet and select the specific worksheet
    sheet = client.open(sheet_name).worksheet(worksheet_name)

    # Read the asset pairs into a DataFrame
    rows = sheet.get_all_values()
    header = rows.pop(0)  # Assumes the first row is the header
    return pd.DataFrame(rows, columns=header)

# Main execution flow
service_account_info = load_service_account_credentials()
client = authenticate_and_get_client(service_account_info)
df_pairs = get_sheet_data(client, "@5mins_phase3_feeder&tracker", "Sheet1")
print(df_pairs.head())  # Example to show some loaded data


  Y Asset X Asset Slope Intercept Z-Score_ Residuals p-value_n p-value_c  \
0                                                                          
1     IEX     PFC                                                          

  Close Price_Correlation % Return_Correlation Z-Score_Ratio  
0                                                             
1                                                             


In [ ]:
from kiteconnect import KiteConnect
import pandas as pd

# Fetch open positions using the Kite Connect API
positions = kite.positions()

# Extract the net positions
open_positions = positions['net']

# Filter out positions with non-zero quantity
filtered_positions = [pos for pos in open_positions if pos['quantity'] != 0]

# Fetch the trade book
trades = kite.trades()

# Create a DataFrame for trades
trades_df = pd.DataFrame(trades)

# Fetch the orders book
orders = kite.orders()

# Create a DataFrame for orders
orders_df = pd.DataFrame(orders)

# Function to get the latest trade entry time for each position
def get_trade_entry_time(tradingsymbol, quantity):
    # Filter trades for the same tradingsymbol and matching quantity
    filtered_trades = trades_df[
        (trades_df['tradingsymbol'] == tradingsymbol) &
        ((trades_df['quantity'] == abs(quantity)) | (trades_df['quantity'] == -abs(quantity)))
    ]
    if not filtered_trades.empty:
        return filtered_trades['order_timestamp'].max()
    # Fallback: approximate entry time using the first or last trade for the tradingsymbol
    symbol_trades = trades_df[trades_df['tradingsymbol'] == tradingsymbol]
    if not symbol_trades.empty:
        return symbol_trades['order_timestamp'].max()
    return None

# Function to get the average price from the orders book
def get_avg_price(tradingsymbol, quantity):
    # Filter orders for the same tradingsymbol and matching quantity
    filtered_orders = orders_df[
        (orders_df['tradingsymbol'] == tradingsymbol) &
        ((orders_df['quantity'] == abs(quantity)) | (orders_df['quantity'] == -abs(quantity)))
    ]
    if not filtered_orders.empty:
        return filtered_orders['average_price'].max()
    # Fallback: approximate average price using any matching orders for the tradingsymbol
    symbol_orders = orders_df[orders_df['tradingsymbol'] == tradingsymbol]
    if not symbol_orders.empty:
        return symbol_orders['average_price'].max()
    return None

# Add entry time and average price to each filtered position
for position in filtered_positions:
    entry_time = get_trade_entry_time(position['tradingsymbol'], position['quantity'])
    avg_price = get_avg_price(position['tradingsymbol'], position['quantity'])
    position['entry_time'] = entry_time
    position['avg_order_price'] = avg_price

# Display the filtered positions with entry times and average order prices
print("Filtered Open Positions (Non-Zero Quantity) with Entry Times and Average Order Prices:")
for position in filtered_positions:
    print(position)

# Convert the filtered positions to a DataFrame
positions_df = pd.DataFrame(filtered_positions)

# Display the DataFrame
positions_df


Filtered Open Positions (Non-Zero Quantity) with Entry Times and Average Order Prices:
{'tradingsymbol': 'JINDALSTEL', 'exchange': 'NSE', 'instrument_token': 1723649, 'product': 'MIS', 'quantity': -5, 'overnight_quantity': 0, 'multiplier': 1, 'average_price': 992.74, 'close_price': 0, 'last_price': 992.65, 'value': 4963.7, 'pnl': 0.4499999999998181, 'm2m': 0.4499999999998181, 'unrealised': 0.4499999999998181, 'realised': 0, 'buy_quantity': 0, 'buy_price': 0, 'buy_value': 0, 'buy_m2m': 0, 'sell_quantity': 5, 'sell_price': 992.74, 'sell_value': 4963.7, 'sell_m2m': 4963.7, 'day_buy_quantity': 0, 'day_buy_price': 0, 'day_buy_value': 0, 'day_sell_quantity': 5, 'day_sell_price': 992.74, 'day_sell_value': 4963.7, 'entry_time': '13:18:55', 'avg_order_price': 992.74}
{'tradingsymbol': 'TATASTEEL', 'exchange': 'NSE', 'instrument_token': 895745, 'product': 'MIS', 'quantity': 30, 'overnight_quantity': 0, 'multiplier': 1, 'average_price': 165.925, 'close_price': 0, 'last_price': 165.55, 'value': -4

,tradingsymbol,exchange,instrument_token,product,quantity,overnight_quantity,multiplier,average_price,close_price,last_price,...,sell_value,sell_m2m,day_buy_quantity,day_buy_price,day_buy_value,day_sell_quantity,day_sell_price,day_sell_value,entry_time,avg_order_price
0,JINDALSTEL,NSE,1723649,MIS,-5,0,1,992.740,0,992.65,...,4963.7,4963.7,0,0.000,0.0,5,992.74,4963.7,13:18:55,992.74
1,TATASTEEL,NSE,895745,MIS,30,0,1,165.925,0,165.55,...,4977.0,4977.0,60,165.925,9955.5,30,165.90,4977.0,13:18:36,165.95


In [ ]:
from kiteconnect import KiteConnect
from requests.exceptions import RequestException, HTTPError
from kiteconnect.exceptions import KiteException
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import datetime
import pandas as pd
import time
from concurrent.futures import ThreadPoolExecutor, as_completed
import statsmodels.api as sm
from statsmodels.tsa.stattools import adfuller

# Function to fetch historical data with retry logic and exponential backoff
def fetch_historical_data_with_retry(instrument_token, from_date_str, to_date_str, interval, max_retries=5):
    retry_delay = 1  # Initial delay in seconds
    for attempt in range(max_retries):
        try:
            data = kite.historical_data(instrument_token=instrument_token,
                                        from_date=from_date_str,
                                        to_date=to_date_str,
                                        interval=interval)
            return data
        except HTTPError as e:
            if e.response.status_code == 429:
                print(f"Rate limit exceeded. Retrying after {retry_delay} seconds...")
                time.sleep(retry_delay)
                retry_delay *= 2  # Double the delay
            else:
                raise e
        except (KiteException, RequestException) as e:
            print(f"Attempt {attempt + 1} failed with error: {e}")
            time.sleep(retry_delay)
        except Exception as e:
            print(f"Non-retryable error: {e}")
            break
    raise Exception("Failed to fetch historical data after multiple retries.")

# Function to fetch data for a single pair
def fetch_data_for_pair(row, open_positions):
    try:
        asset1 = row['Y Asset']
        asset2 = row['X Asset']

        # Find open positions details
        pos_asset1 = next((p for p in open_positions if p['tradingsymbol'] == asset1), None)
        pos_asset2 = next((p for p in open_positions if p['tradingsymbol'] == asset2), None)

        if not pos_asset1 or not pos_asset2:
            print(f"Warning: Incomplete open positions found for the pair {asset1}-{asset2}.")
            return None

        asset1_entry_price = pos_asset1.get('avg_order_price', pos_asset1['average_price']) if pos_asset1 else None
        asset1_entry_time = pos_asset1.get('entry_time') if pos_asset1 else None
        asset1_token = pos_asset1.get('instrument_token') if pos_asset1 else None

        asset2_entry_price = pos_asset2.get('avg_order_price', pos_asset2['average_price']) if pos_asset2 else None
        asset2_entry_time = pos_asset2.get('entry_time') if pos_asset2 else None
        asset2_token = pos_asset2.get('instrument_token') if pos_asset2 else None

        # Ensure both entry times are not None
        if not asset1_entry_time or not asset2_entry_time:
            print(f"Warning: Missing entry time for the pair {asset1}-{asset2}")
            return None

        # Fetch historical data
        asset1_data = fetch_historical_data_with_retry(
            instrument_token=asset1_token,
            from_date_str=start_date_str,
            to_date_str=end_date_str,
            interval='5minute'
        ) if asset1_token else []
        asset2_data = fetch_historical_data_with_retry(
            instrument_token=asset2_token,
            from_date_str=start_date_str,
            to_date_str=end_date_str,
            interval='5minute'
        ) if asset2_token else []

        asset1_df = pd.DataFrame(asset1_data)
        asset2_df = pd.DataFrame(asset2_data)

        # Limit data to the last 250 data points
        asset1_df = asset1_df.tail(250)
        asset2_df = asset2_df.tail(250)

        # Replace the entry time close prices with actual trade prices
        if asset1_entry_price:
            asset1_entry_candle_time = asset1_df[asset1_df['date'] <= asset1_entry_time]['date'].max()
            asset1_df.loc[asset1_df['date'] == asset1_entry_candle_time, 'close'] = asset1_entry_price

        if asset2_entry_price:
            asset2_entry_candle_time = asset2_df[asset2_df['date'] <= asset2_entry_time]['date'].max()
            asset2_df.loc[asset2_df['date'] == asset2_entry_candle_time, 'close'] = asset2_entry_price

        # Combine the close prices and timestamps
        combined_df = pd.merge(
            asset1_df[['date', 'close']].rename(columns={'date': 'timestamp1', 'close': 'asset1_close'}),
            asset2_df[['date', 'close']].rename(columns={'date': 'timestamp2', 'close': 'asset2_close'}),
            left_on='timestamp1', right_on='timestamp2', how='inner'
        )

        # Optionally, drop one of the timestamps if they are identical and you want a cleaner DataFrame
        if 'timestamp1' in combined_df.columns and 'timestamp2' in combined_df.columns:
            # Check if timestamps from both assets are identical
            if all(combined_df['timestamp1'] == combined_df['timestamp2']):
                combined_df.drop('timestamp2', axis=1, inplace=True)
                combined_df.rename(columns={'timestamp1': 'timestamp'}, inplace=True)

        return {
            'pair': row['Y Asset'] + '-' + row['X Asset'],
            'data': combined_df,
            'asset1_entry_time': asset1_entry_time,
            'asset2_entry_time': asset2_entry_time
        }

    except Exception as e:
        print(f"Failed to fetch data for pair {row['Y Asset']}-{row['X Asset']}: {e}")
        return None

# Statistical Analysis Functions
def perform_linear_regression(Y, X):
    # Ensure X and Y have matching indices
    X = X.reindex(Y.index).dropna()  # Reindex X to match Y and drop any resulting NaNs
    Y = Y.reindex(X.index).dropna()  # Ensure Y matches the new X index after potentially dropping values in X

    if X.empty or Y.empty:
        return None  # Return None if either X or Y becomes empty after reindexing

    X = sm.add_constant(X)  # Adds a constant term to the predictor
    model = sm.OLS(Y, X).fit()

    # Calculate needed metrics
    slope = model.params[1]
    intercept = model.params[0]
    residuals = model.resid
    std_error_intercept = model.bse[0]
    std_dev_residuals = residuals.std()
    error_ratio = std_error_intercept / std_dev_residuals

    # Calculate the Z-Score for the latest residual value
    latest_residual = residuals.iloc[-1] if len(residuals) > 0 else None
    z_score = latest_residual / std_dev_residuals if latest_residual is not None else None

    # ADF Test on residuals
    adf_result_n = adfuller(residuals, regression='n')
    adf_result_c = adfuller(residuals, regression='c')

    return {
        "slope": slope,
        "intercept": intercept,
        "residuals": residuals,
        "std_error_intercept": std_error_intercept,
        "std_dev_residuals": std_dev_residuals,
        "error_ratio": error_ratio,
        "z_score": z_score,
        "adf_statistic_n": adf_result_n[0],
        "p_value_n": adfuller(residuals, regression='n')[1],
        "adf_statistic_c": adfuller(residuals, regression='c')[0],
        "p_value_c": adfuller(residuals, regression='c')[1]
    }

def additional_metrics(asset1_df, asset2_df):
    combined_df = pd.concat([asset1_df, asset2_df], axis=1)
    combined_df.columns = ['asset1_close', 'asset2_close']  # Rename columns if necessary
    combined_df.dropna(inplace=True)  # Ensure data alignment by dropping NaN values
    combined_df['asset1_returns'] = combined_df['asset1_close'].pct_change()
    combined_df['asset2_returns'] = combined_df['asset2_close'].pct_change()
    combined_df.dropna(inplace=True)  # Drop NaNs introduced by pct_change()
    combined_df['ratio'] = combined_df['asset1_close'] / combined_df['asset2_close']  # Calculate the ratio of asset1 to asset2

    return {
        "correlation_price": combined_df['asset1_close'].corr(combined_df['asset2_close']),
        "correlation_returns": combined_df['asset1_returns'].corr(combined_df['asset2_returns']),
        "z_score_ratio": (combined_df['ratio'].iloc[-1] - combined_df['ratio'].mean()) / combined_df['ratio'].std()
    }

# Function to calculate Z-scores at entry time
def calculate_z_scores_at_entry(combined_df, entry_time, ratio_column='ratio', residuals_column='residuals'):
    if entry_time is None:
        return None, None, None
    entry_candle_time = combined_df[combined_df['timestamp'] <= entry_time]['timestamp'].max()
    ratio_entry_value = combined_df.loc[combined_df['timestamp'] == entry_candle_time, ratio_column].values[0]
    residuals_entry_value = combined_df.loc[combined_df['timestamp'] == entry_candle_time, residuals_column].values[0]

    ratio_z_score_entry = (ratio_entry_value - combined_df[ratio_column].mean()) / combined_df[ratio_column].std()
    residuals_z_score_entry = residuals_entry_value / combined_df[residuals_column].std()

    return ratio_z_score_entry, residuals_z_score_entry, entry_candle_time

# Fetch open positions using the Kite Connect API
positions = kite.positions()
open_positions = positions['net']

# Fetch the trade book
trades = kite.trades()
trades_df = pd.DataFrame(trades)

# Fetch the orders book
orders = kite.orders()
orders_df = pd.DataFrame(orders)

# Function to get the latest trade entry time for each position
def get_trade_entry_time(tradingsymbol, quantity):
    filtered_trades = trades_df[
        (trades_df['tradingsymbol'] == tradingsymbol) &
        ((trades_df['quantity'] == abs(quantity)) | (trades_df['quantity'] == -abs(quantity)))
    ]
    if not filtered_trades.empty:
        return filtered_trades['order_timestamp'].max()
    symbol_trades = trades_df[trades_df['tradingsymbol'] == tradingsymbol]
    if not symbol_trades.empty:
        return symbol_trades['order_timestamp'].max()
    return None

# Function to get the average price from the orders book
def get_avg_price(tradingsymbol, quantity):
    filtered_orders = orders_df[
        (orders_df['tradingsymbol'] == tradingsymbol) &
        ((orders_df['quantity'] == abs(quantity)) | (orders_df['quantity'] == -abs(quantity)))
    ]
    if not filtered_orders.empty:
        return filtered_orders['average_price'].max()
    symbol_orders = orders_df[orders_df['tradingsymbol'] == tradingsymbol]
    if not symbol_orders.empty:
        return symbol_orders['average_price'].max()
    return None

# Add entry time and average price to each open position
for position in open_positions:
    entry_time = get_trade_entry_time(position['tradingsymbol'], position['quantity'])
    avg_price = get_avg_price(position['tradingsymbol'], position['quantity'])
    position['entry_time'] = entry_time
    position['avg_order_price'] = avg_price if avg_price else position['average_price']

sheet_name = '@5mins_phase3_feeder&tracker'
sheet = client.open(sheet_name)
worksheet = sheet.get_worksheet(0)

# Read data from the sheet
pairs_data = worksheet.get_all_records()
pairs_df = pd.DataFrame(pairs_data)

# Calculate the date range for the last 8 days
end_date = datetime.datetime.today().date()
start_date = end_date - datetime.timedelta(days=8)
start_date_str = start_date.strftime('%Y-%m-%d')
end_date_str = end_date.strftime('%Y-%m-%d')

# Match pairs with open positions
open_positions_set = set([pos['tradingsymbol'] for pos in open_positions])
matched_pairs = []

for _, row in pairs_df.iterrows():
    asset1 = row['Y Asset']
    asset2 = row['X Asset']

    if asset1 in open_positions_set and asset2 in open_positions_set:
        matched_pairs.append(f"{asset1}-{asset2}")

# Output matched pairs
print("Matched pairs from Google Sheet with open positions:")
print(matched_pairs if matched_pairs else "No matching pairs found.")

# Fetch data for all matched pairs
pair_dataframes = {}
with ThreadPoolExecutor(max_workers=2) as executor:  # Adjust max_workers based on your API's rate limit
    futures = [executor.submit(fetch_data_for_pair, row, open_positions) for _, row in pairs_df.iterrows() if f"{row['Y Asset']}-{row['X Asset']}" in matched_pairs]
    for future in as_completed(futures):
        result = future.result()
        if result is not None:
            pair_dataframes[result['pair']] = result

# Perform statistical analysis
results = []
for pair_name, data in pair_dataframes.items():
    asset1_name, asset2_name = pair_name.split('-')
    combined_df = data['data']
    asset1_entry_time = data['asset1_entry_time']
    asset2_entry_time = data['asset2_entry_time']

    # Perform the regression analysis
    results1 = perform_linear_regression(combined_df['asset1_close'], combined_df['asset2_close'])
    results2 = perform_linear_regression(combined_df['asset2_close'], combined_df['asset1_close'])

    # Calculate additional metrics
    additional_results = additional_metrics(combined_df['asset1_close'], combined_df['asset2_close'])

    # Add the additional metrics to the regression results
    results1.update(additional_results)
    results2.update(additional_results)

    # Determine the preferred set of results
    preferred_results = results1 if results1['error_ratio'] < results2['error_ratio'] else results2
    preferred_results.update({"Y": asset1_name if results1['error_ratio'] < results2['error_ratio'] else asset2_name,
                              "X": asset2_name if results1['error_ratio'] < results2['error_ratio'] else asset1_name})

    # Calculate Z-Scores at the trade entry time
    combined_df['residuals'] = preferred_results['residuals']
    combined_df['ratio'] = combined_df['asset1_close'] / combined_df['asset2_close']

    entry_time = asset1_entry_time if preferred_results['Y'] == asset1_name else asset2_entry_time
    ratio_z_score_entry, residuals_z_score_entry, entry_candle_time = calculate_z_scores_at_entry(combined_df, entry_time)

    preferred_results.update({
        "z_score_ratio_at_entry": ratio_z_score_entry,
        "z_score_residuals_at_entry": residuals_z_score_entry,
        "entry_candle_time": entry_candle_time
    })

    results.append(preferred_results)

# Display the chosen results
for result in results:
    print(f"Analysis for pair {result['Y']} (Y) vs {result['X']} (X) with lower error ratio:")
    print("Slope:", result["slope"])
    print("Intercept:", result["intercept"])
    print("All Residual Values:\n", result["residuals"])
    print("Standard Error of the Intercept:", result["std_error_intercept"])
    print("Standard Deviation of the Residuals:", result["std_dev_residuals"])
    print("Error Ratio:", result["error_ratio"])
    print("Z-Score:", result.get("z_score", "Not Calculated"))
    print("ADF Statistic (no trend):", result["adf_statistic_n"])
    print("P-Value (no trend):", result["p_value_n"])
    print("ADF Statistic (constant only):", result["adf_statistic_c"])
    print("P-Value (constant only):", result["p_value_c"])
    print("Correlation (Price):", result["correlation_price"])
    print("Correlation (Returns):", result["correlation_returns"])
    print("Z-Score Ratio:", result["z_score_ratio"])
    print("Z-Score Ratio at Entry Time:", result["z_score_ratio_at_entry"])
    print("Z-Score Residuals at Entry Time:", result["z_score_residuals_at_entry"])
    print("Entry Candle Time used for Z-Score calculation:", result["entry_candle_time"])
    print("\n")


Matched pairs from Google Sheet with open positions:
['TATASTEEL-JINDALSTEL']
Analysis for pair TATASTEEL (Y) vs JINDALSTEL (X) with lower error ratio:
Slope: 0.07182737951699109
Intercept: 95.17854714888533
All Residual Values:
 0      1.291622
1      1.612891
2      1.534439
3      1.391901
4      1.445771
         ...   
245   -1.053693
246   -1.164467
247   -1.258960
248   -0.811714
249   -1.154531
Length: 250, dtype: float64
Standard Error of the Intercept: 1.7405580813075499
Standard Deviation of the Residuals: 0.7875820013335734
Error Ratio: 2.2100023595769707
Z-Score: -1.4659182607170513
ADF Statistic (no trend): -2.417171697686473
P-Value (no trend): 0.015090374113306396
ADF Statistic (constant only): -2.4104745630473685
P-Value (constant only): 0.13880183938621837
Correlation (Price): 0.9298898511070326
Correlation (Returns): 0.6628699004206088
Z-Score Ratio: -1.6752608628891135
Z-Score Ratio at Entry Time: -1.5638620757804982
Z-Score Residuals at Entry Time: -1.0306400782955

In [ ]:
import zipfile
import os

# Create a folder to store individual CSV files
output_dir = "/content/pair_data"
os.makedirs(output_dir, exist_ok=True)

# Save each pair's data as a separate CSV file
for pair_name, data_dict in pair_dataframes.items():
    data = data_dict['data']  # Access the DataFrame
    output_file = os.path.join(output_dir, f"{pair_name}.csv")
    data.to_csv(output_file, index=False)

# Zip all CSV files
zip_file_path = "/content/pair_data.zip"
with zipfile.ZipFile(zip_file_path, "w") as z:
    for csv_file in os.listdir(output_dir):
        z.write(os.path.join(output_dir, csv_file), csv_file)




In [ ]:
# Open the specific Google Sheet and worksheet
sheet_name = '@5mins_phase3_feeder&tracker'
sheet = client.open(sheet_name)
worksheet = sheet.worksheet("Sheet2")

# Prepare the headers and data for the worksheet
headers = [
    "Y Asset", "X Asset", "Slope", "Intercept", "Z-Score_ Residuals",
    "p-value_n", "p-value_c", "Close Price_Correlation", "% Return_Correlation",
    "Z-Score_Ratio", "Z-Score Ratio at Entry Time", "Z-Score Residuals at Entry Time",
    "2nd Z-Score_ Entry Time", "Z_Ratio_Diff", "Z_Residuals_Diff"
]

# Function to safely compute the absolute difference
def calculate_absolute_difference(current, entry):
    try:
        return abs(current) - abs(entry)
    except TypeError:
        return None

# Convert the results into a list of lists
results_data = []
for result in results:
    z_ratio_diff = calculate_absolute_difference(result['z_score_ratio'], result['z_score_ratio_at_entry'])
    z_residuals_diff = calculate_absolute_difference(result['z_score'], result['z_score_residuals_at_entry'])

    results_data.append([
        result['Y'], result['X'], result['slope'], result['intercept'], result['z_score'],
        result['p_value_n'], result['p_value_c'], result['correlation_price'], result['correlation_returns'],
        result['z_score_ratio'], result['z_score_ratio_at_entry'], result['z_score_residuals_at_entry'],
        result['entry_candle_time'].strftime('%Y-%m-%d %H:%M:%S') if isinstance(result['entry_candle_time'], pd.Timestamp) else result['entry_candle_time'],
        z_ratio_diff, z_residuals_diff
    ])

# Read existing data, if any, to retain it
existing_data = worksheet.get_all_values()

# If no existing data, add headers first
if not existing_data:
    worksheet.append_row(headers)

# Add a blank row if data already exists
last_row_index = len(existing_data) + 1 if existing_data else 1
worksheet.insert_row([''] * len(headers), index=last_row_index + 1)

# Append new results data
worksheet.append_rows(results_data, table_range=f"A{last_row_index + 2}")

# Debugging output
print("Results Data Debugging:")
for row in results_data:
    print(row)


Results Data Debugging:
['TATASTEEL', 'JINDALSTEL', 0.07182737951699109, 95.17854714888533, -1.4659182607170513, 0.015090374113306396, 0.13880183938621837, 0.9298898511070326, 0.6628699004206088, -1.6752608628891135, -1.5638620757804982, -1.0306400782955984, '2024-05-15 14:50:00', 0.11139878710861528, 0.4352781824214529]
